# Exploring an XCLBIN file with PYNQ

In this notebook we explore the ways that PYNQ can be used to see what's in an xclbin file.

To start with we'll download a bitstream from the internet using the `urllib` library that comes as part of the standard library. The bitstream is from the kernel optimisation notebooks but any other accessible xclbin file could be used. Any xclbin file generated from Vitis is usable from PYNQ without any changes

In [1]:
import urllib.request

with urllib.request.urlopen('https://bit.ly/38ukSXG') as source, \
     open('downloaded.xclbin', 'wb') as sink:
    data = source.read()
    sink.write(data)

The starting point of all xclbin files is the `Overlay` class. To explore the bitstream without downloading it we can pass the `download=False` parameters to the constructor. This won't let us instantiate drivers but we can still explore the data-structures contained in the xclbin file.

In [2]:
import pynq

ol = pynq.Overlay('downloaded.xclbin', download=False)

To list everything in a bitstream inside Jupyter you can use the `?` operator to display the instance's documentation. This is dynamically generated based on the metadata encoded within the xclbin file.

In [3]:
ol?

Type:            Overlay
String form:     <pynq.overlay.Overlay object at 0x7f50cb89b990>
File:            /scratch/pynq-testing/ogden/conda/lib/python3.7/site-packages/pynq/overlay.py
Docstring:      
Default documentation for overlay downloaded.xclbin. The following
attributes are available on this overlay:

IP Blocks
----------
vadd_wide_multibanks_1 : pynq.overlay.DefaultIP
vadd_wide_1          : pynq.overlay.DefaultIP
krnl_stream_vadd_1   : pynq.overlay.DefaultIP
krnl_stream_vmult_1  : pynq.overlay.DefaultIP

Hierarchies
-----------
None

Interrupts
----------
None

GPIO Outputs
------------
None

Memories
------------
bank0                : Memory
bank1                : Memory
bank2                : Memory
dc_0                 : Stream
Class docstring:
This class keeps track of a single bitstream's state and contents.

The overlay class holds the state of the bitstream and enables run-time
protection of bindlings.

Our definition of overlay is: "post-bitstream configurable design

More details on each object in the bitstream can be found from two dictionaries - the `ip_dict` containing the IP details and the `mem_dict` containing the memories and streams.

## The IP Dictionary

The `ip_dict` contains a lot of information so a good starting point is to just look at the keys of the dictionary.

In [4]:
ol.ip_dict.keys()

dict_keys(['vadd_wide_multibanks_1', 'vadd_wide_1', 'krnl_stream_vadd_1', 'krnl_stream_vmult_1'])

This exactly matches what was seen in the overlay documentation. We can now look in detail at one particular entry - the `vadd_wide_1`. If you are in JupyterLab this will create a searchable, hierarchical view of the dictionary.

In [5]:
ol.ip_dict['vadd_wide_1']

{'phys_addr': 25165824,
 'addr_range': 4096,
 'type': 'xilinx.com:hls:vadd_wide:1.0',
 'fullpath': 'vadd_wide_1',
 'registers': {'CTRL': {'address_offset': 0,
   'access': 'read-write',
   'size': 4,
   'description': 'OpenCL Control Register',
   'type': 'unsigned int',
   'id': None,
   'fields': {'AP_START': {'access': 'read-write',
     'bit_offset': 0,
     'bit_width': 1,
     'description': 'Start the accelerator'},
    'AP_DONE': {'access': 'read-only',
     'bit_offset': 1,
     'bit_width': 1,
     'description': 'Accelerator has finished - cleared on read'},
    'AP_IDLE': {'access': 'read-only',
     'bit_offset': 2,
     'bit_width': 1,
     'description': 'Accelerator is idle'},
    'AP_READY': {'access': 'read-only',
     'bit_offset': 3,
     'bit_width': 1,
     'description': 'Accelerator is ready to start next computation'},
    'AUTO_RESTART': {'access': 'read-write',
     'bit_offset': 7,
     'bit_width': 1,
     'description': 'Restart the accelerator automatical

Most of the dictionary is dedicated to the register map where each register and field is enumerated. This information is used by PYNQ to reconstruct the function call and to correctly program the accelerator as a function call. Other useful entries in the dictionary are `phys_addr` which gives the address of the control logic and `type` which identfies the IP core of the accelerator.

## The Memory Dictionary

The memory dictionary contains information on the memory banks and streams in the system. Again we can dump the keys before having a closer look.

In [6]:
ol.mem_dict.keys()

dict_keys(['bank0', 'bank1', 'bank2', 'bank3', 'PLRAM[0]', 'PLRAM[1]', 'PLRAM[2]', 'dc_0'])

In most cases the xclbin file will contain information on all memory banks in the platform and this information carries through into the dictionary. This means that not all banks in the dictionary are actually used or accessible in the design. There is a `used` key in each sub-dictionary to determine whether a bank is used in the bitstream.

In [7]:
ol.mem_dict['bank3']

{'raw_type': 1,
 'used': 0,
 'base_address': 481036337152,
 'size': 17179869184,
 'type': 'DDR4',
 'streaming': False,
 'idx': 3}

The `base_address` and `size` fields can be useful for debugging allocated buffers and ensuring there is sufficient memory to run an application. The rest of the sub-dictionary is primarily for the internals of PYNQ to be able to interact with the hardware correctly.

For streams the `streaming` key will be to `True` and a different set of entries are listed in the dictionary:

In [8]:
ol.mem_dict['dc_0']

{'raw_type': 9,
 'used': 1,
 'flow_id': 0,
 'route_id': 0,
 'type': 'AXI Stream',
 'streaming': True,
 'idx': 7}

Again, the entries are primarily to support the low-levels of PYNQ and generally won't be needed in user code except to enumerate the streams in the system. Higher-level contructs are available through the Overlay class once the bitstream is loaded and these will be dicussed in the _Memories and Streams_ notebook in the kernel optimisation folder.

Copyright (C) 2020 Xilinx, Inc